### Measure step length from depth image

In [1]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [2]:
import os
import glob
import functools

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

import modules.images as im

In [3]:
load_dir = os.path.join(os.path.sep, 'Users', 'andrewhynes', 'MEGA', 'Data',\
                        'Czarnuch images', 'Depth data', '14-12-16_trial-004_depth16bit')

In [4]:
file_paths = glob.glob(os.path.join(load_dir, '*.png'))

In [5]:
x_res, y_res = 640, 480

f_xz = 1.11146664619446
f_yz = 0.833599984645844

f_xz_new = 1.122133  
f_yz_new =  0.84176

# f_xz = f_xz_new
# f_yz = f_yz_new


In [6]:
real_world = functools.partial(im.image_coords_to_real, x_res, y_res, f_xz, f_yz)

In [7]:
def foo(img, x_i, y_i, x_f, y_f):
    
    point_i = real_world(img, x_i, y_i)
    point_f = real_world(img, x_f, y_f)
    
    return np.linalg.norm(point_i - point_f)

In [8]:
frame_1, frame_2, frame_3 = 233, 245, 260

# Depth image (convert to cm)
img_1 = cv2.imread(file_paths[frame_1], cv2.IMREAD_UNCHANGED) / 10
img_2 = cv2.imread(file_paths[frame_2], cv2.IMREAD_UNCHANGED) / 10
img_3 = cv2.imread(file_paths[frame_3], cv2.IMREAD_UNCHANGED) / 10

In [9]:
d1 = foo(img_1, 172, 390, 295, 390)
d2 = foo(img_2, 300, 400, 425, 390)
d3 = foo(img_3, 420, 395, 565, 390)

[d1, d2, d3]

[61.81560268272917, 64.75517285837184, 63.984231816584135]

In [10]:
dataframe_dir = os.path.join('..', 'data', 'kinect', 'best pos')
file_name = '2014-12-16_P005_Post_004.pkl'


In [11]:
df_head_feet = pd.read_pickle(os.path.join(dataframe_dir, file_name))

In [12]:
point_1L, point_1R = df_head_feet.loc[frame_1][['L_FOOT', 'R_FOOT']]
point_2L, point_2R = df_head_feet.loc[frame_2][['L_FOOT', 'R_FOOT']]
point_3L, point_3R = df_head_feet.loc[frame_3][['L_FOOT', 'R_FOOT']]

image_pos_1L = np.round(im.real_to_proj(point_1L, x_res, y_res, f_xz, f_yz))[:2].astype(int)
image_pos_1R = np.round(im.real_to_proj(point_1R, x_res, y_res, f_xz, f_yz))[:2].astype(int)

image_pos_2L = np.round(im.real_to_proj(point_2L, x_res, y_res, f_xz, f_yz))[:2].astype(int)
image_pos_2R = np.round(im.real_to_proj(point_2R, x_res, y_res, f_xz, f_yz))[:2].astype(int)

image_pos_3L = np.round(im.real_to_proj(point_3L, x_res, y_res, f_xz, f_yz))[:2].astype(int)
image_pos_3R = np.round(im.real_to_proj(point_3R, x_res, y_res, f_xz, f_yz))[:2].astype(int)

In [13]:
d1_kinect = np.linalg.norm(point_1L - point_1R)
d2_kinect = np.linalg.norm(point_2L - point_2R)
d3_kinect = np.linalg.norm(point_3L - point_3R)

[d1_kinect, d2_kinect, d3_kinect]

[51.11247999168109, 51.02569721307099, 56.7126316925956]

In [14]:
plt.figure()
plt.imshow(img_1)
plt.scatter(image_pos_1L[0], image_pos_1L[1], c='w', edgecolors='k')
plt.scatter(image_pos_1R[0], image_pos_1R[1], c='w', edgecolors='k')
plt.title('Frame {}'.format(frame_1))

plt.figure()
plt.imshow(img_2)
plt.scatter(image_pos_2L[0], image_pos_2L[1], c='w', edgecolors='k')
plt.scatter(image_pos_2R[0], image_pos_2R[1], c='w', edgecolors='k')
plt.title('Frame {}'.format(frame_2))

plt.figure()
plt.imshow(img_3)
plt.scatter(image_pos_3L[0], image_pos_3L[1], c='w', edgecolors='k')
plt.scatter(image_pos_3R[0], image_pos_3R[1], c='w', edgecolors='k')
plt.title('Frame {}'.format(frame_3))

plt.show()

In [15]:
foo(img_3, 404, 389, 528, 389)

55.499953291813675

In [16]:
foo(img_3, 388, 400, 510, 400)

61.216616040002584

In [17]:
foo(img_3, 255, 400, 375, 393, )

58.93280119505982

### Iterate to find best $f_{xz}$ and $f_{yz}$

In [21]:
point_proj_1L = np.round(im.real_to_proj(point_1L, x_res, y_res, f_xz, f_yz))
point_proj_1R = np.round(im.real_to_proj(point_1R, x_res, y_res, f_xz, f_yz))

max_error = np.inf

for f_xz_test in np.linspace(0, 2, num=200):
    for f_yz_test in np.linspace(0, 2, num=200):
        
        
        point_1L_test = im.proj_to_real(point_proj_1L, x_res, y_res, f_xz_test, f_yz_test)
        point_1R_test = im.proj_to_real(point_proj_1R, x_res, y_res, f_xz_test, f_yz_test)

        d1_kinect = np.linalg.norm(point_1L_test - point_1R_test)
        
        d_test_1L = np.linalg.norm(point_1L_test - point_1L)
        d_test_1R = np.linalg.norm(point_1R_test - point_1R)
                
        error = abs(d1 - d1_kinect)
        
        if error < max_error:
            
            max_error = error
            f_xz_best = f_xz_test
            f_yz_best = f_yz_test
            
            d1_kinect_best = d1_kinect
            point_1L_best = point_1L_test
            point_1R_best = point_1R_test
            
d1_kinect_best

61.8154275636409

In [22]:
image_pos_1L = np.round(im.real_to_proj(point_1L_best, x_res, y_res, f_xz_best, f_yz_best))[:2].astype(int)
image_pos_1R = np.round(im.real_to_proj(point_1R_best, x_res, y_res, f_xz_best, f_yz_best))[:2].astype(int)